# Perform OCR operation (read image)

In [7]:
import re
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd
from collections import namedtuple
from tkinter import *
from tkinter import filedialog

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

root=Tk()
fileName =filedialog.askopenfilename(initialdir="/img",title="Select a picture as raw image",filetypes=(("jpg files","*.jpg"),("All files", "*.*")))
root.destroy()

img = cv2.imread(fileName)

d = pytesseract.image_to_data(img, output_type=Output.DICT)

text = pytesseract.image_to_string(img)
print(text)

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

img = cv2.resize(img, (800,960))
cv2.imshow('img', img)
cv2.waitKey(0)

ALD AUTOMOTIVE PLTD.

ADISHAKTI CARS PVT. LTD.

» Passenger Car Dealer

Workafella Business Centre 150/1, Infantry Road,

Opp. Commissioner Office, Vasanth Nagar, Bangalore 560001

BENGALURU NORTH, BENGALURU

BENGALURU, 560001
Karnataka(29)

Phone No (Res,Off,Mob): , , 8657489117
Customer GSTIN : 29AAFCA0924K1ZN
AIC Code : 1-150DE1KN
Place of Supply: 29-Karnataka

pete |Prm2]4F4oR2
TATA MOTORS

CIN No U50102KA2010PTCO55116

TAX INVOICE

Issued under GST Invoice Rules Original :For Recipient

Duplicate :For Supplier

Invoice No : IATIEN1819013536

Invoice Date :15/02/2019

Model : TATA HEXA

Chassis No : MAT614406JRF06620
Insurance Co : BAJAJ ALLIANZ INS. CO. LTD.
Kms. : 10121

Vehicle Regn. No : KA0Q1MS1164

Job Card No. : JC-AtiCar-BP1-1819-012912
Job Card Date : 15/02/2019

Service Request Type : Second Free Service
Customer P.O. No - Date :

  

 

 

   
  

 

 

 

 

 

 

   

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

   

   

 

<n, Payment Method : CREDIT
v ES * Warran

-1

# Extraction of data

In [2]:
invNo = re.compile(r'Inv(\w\w\w\w|oice) (\w\w|No) : (\w*)')
inv1 = invNo.search(text)
inv_no = inv1.group(3)

invDate = re.compile(r'Inv(\w\w\w\w|oice) (\w\w\w\w|Date) :(\d*/\d*/\d*)')
inv2 = invDate.search(text)
inv_date = inv2.group(3)

chassisNo = re.compile(r'Ch(\w\w\w\w\w|assis) (\w\w|No) : (\w*)')
inv3 = chassisNo.search(text)
chassis_no = inv3.group(3)

k = re.compile(r'Kms. : (\d*)')
kk = k.search(text)
kms = kk.group(1)

v = re.compile(r'V(\w\w\w\w\w\w|ehicle) R(\w\w\w|egn). (\w\w|No) : (\w*)')
vrn = v.search(text)
vehicle_no = vrn.group(4)

jcn = re.compile(r'J(\w\w|ob) (\w\w\w\w|Card) (\w\w|No). : (\w*-\w*-\w*-\w*-\w*)')
jcN = jcn.search(text)
job_card_no = jcN.group(4)

jcd = re.compile(r'J(\w\w|ob) (\w\w\w\w|Card) (\w\w\w\w|Date) : (\d*/\d*/\d*)')
jcD = jcd.search(text)
job_card_date = jcD.group(4)

sr = re.compile(r'S(\w\w\w\w\w\w|ervice) (\w\w\w\w\w\w\w|Request) (\w\w\w\w|Type) : (R\w\w\w\w\w\w R\w\w\w\w\w\w|C\w\w\w\w\w\w\w|P\w\w\w S\w\w\w\w\w\w|S\w\w\w\w\w F\w\w\w S\w\w\w\w\w\w)')
srt = sr.search(text)
service_request_type = srt.group(4)

pm = re.compile(r'P(\w\w\w\w\w\w|ayment) M(|\w\w\w\w\w|ethod) : (C\w\w\w\w\w|C\w\w\w)')
payment = pm.search(text)
payment_method = payment.group(3)

w = re.compile(r'W(\w\w\w\w\w\w\w|arranty) E(\w\w\w\w\w\w|xpired) :(N|Y|\w)')
we = w.search(text)
warranty_expired = we.group(3)

gt = re.compile(r'G(\w\w\w\w|rand) T(\w\w\w\w|otal) : (\d(,)?\d*(.)?\d*)')
gtl = gt.search(text)
grand_total = gtl.group(3)

In [3]:
INVOICE = namedtuple('INVOICE', 'InvoiceNo InvoiceDate ChassisNo Kms VehicleNo JobCardNo JobCardDate ServiceRequestType PaymentMethod WarrantyExpired GrandTotal')
lineItem = []
lineItem.append(INVOICE(inv_no,inv_date,chassis_no,kms,vehicle_no,job_card_no,job_card_date,service_request_type,payment_method,warranty_expired,grand_total))


In [4]:
df = pd.DataFrame(lineItem)
df

,InvoiceNo,InvoiceDate,ChassisNo,Kms,VehicleNo,JobCardNo,JobCardDate,ServiceRequestType,PaymentMethod,WarrantyExpired,GrandTotal
0,IATIEN1819013538,15/02/2019,MAT629301HKF42433,16000,KA01MQ6424,JC-AtiCar-BP1-1819-012923,15/02/2019,Running Repairs,CREDIT,N,"1,416.00"


In [5]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['JobCardDate'] = pd.to_datetime(df['JobCardDate'])
df

,InvoiceNo,InvoiceDate,ChassisNo,Kms,VehicleNo,JobCardNo,JobCardDate,ServiceRequestType,PaymentMethod,WarrantyExpired,GrandTotal
0,IATIEN1819013538,2019-02-15,MAT629301HKF42433,16000,KA01MQ6424,JC-AtiCar-BP1-1819-012923,2019-02-15,Running Repairs,CREDIT,N,"1,416.00"


In [6]:
# df.to_csv('Invoice Info.csv', mode='w', header=True)
df.to_csv('Invoice Info.csv', mode='a', header=False)